In [181]:
import pandas as pd
import itertools


In [182]:
df = pd.read_csv('data_speech_all.csv', sep='\t', encoding='utf-8')
df.head()

,Unnamed: 0,episode,timestamp,player,character,speech,segmented,syntax,intent
0,0,S1E601,captain_election,1号,民,1号预言家,1 号 预言家,预言家 核心关系 主谓关系 主谓关系 主谓关系 主谓关系,defense
1,1,S1E601,captain_election,1号,民,昨天晚上查杀是5号牌,昨天晚上 查杀 是 5 号牌,是 核心关系 查杀 主谓关系 牌 主谓关系,attack
2,2,S1E601,captain_election,1号,民,我的警徽流先验9号这张牌，帮大家正一正场上的风气，如果是查杀的话就直接走掉了,我 的 警徽流 先验 9 号 这 张牌 ， 帮 大家 正一正 场上 的 风气 ， 如果 是 ...,帮 是 牌 主谓关系 风气 查杀,to_check
3,3,S1E601,captain_election,1号,民,然后再验呢，再验3号这张牌，3号最近进步太大了，好吧,然后 再验 呢 ， 再验 3 号 这 张牌 ， 3 号 最近 进步 太大 了 ， 好 吧,验 进步 号 好 牌 主谓关系,to_check
4,4,S1E601,captain_election,1号,民,我觉得你们都是一个水平线的玩家，我先验9再验3,我 觉得 你们 都 是 一个 水平线 的 玩家 ， 我 先验 9 再验 3,觉得 验 我 你们 是 玩家,to_check


In [183]:
df_test = df[['timestamp','player','character','speech','segmented','syntax']][df.episode=="S1E102"]
df_test.head()

,timestamp,player,character,speech,segmented,syntax
3384,captain_election,5号,民,警上这么多人，只有四个人没竞选好吧，那我认警上应该是有两批狼左右,警上 这么 多人 ， 只有 四个 人 没 竞选 好 吧 ， 那 我认 警上 应该 是 有 两...,竞选 是 人 人 警上 有
3385,captain_election,5号,民,呃很多村民竞选是不好的事，因为村民不在警下，其实给狼队提供了刀神的机会,呃 很多 村民 竞选 是 不好 的 事 ， 因为 村民 不 在 警下 ， 其实 给 狼队 提...,是 不在 呃 村民 事 警下
3386,captain_election,5号,民,因为有些神是不敢上警的,因为 有些 神是 不敢 上警 的,是 核心关系 神 主谓关系 上警 主谓关系
3387,captain_election,5号,民,当然你狼人可以赌反心态,当然 你 狼 人 可以 赌反 心态,赌 反 狼人 主谓关系 心态 主谓关系
3388,captain_election,5号,民,所以我建议以后村民不要乱起跳，像我这种强民确实是下次不跳了,所以 我 建议 以后 村民 不要 乱 起跳 ， 像 我 这种 强民 确实 是 下次 不 跳 了,建议 核心关系 村民 强民 起跳 跳


In [184]:
df_train = df[['segmented','intent']][df.episode!="S1E102"]

X_train = df_train.segmented
y_train = df_train.intent

In [185]:
# label intent by tuned models
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC


# with open("model_SVM.pickle","rb") as m:
#     model_svm = pickle.load(m)
# with open("tfidf.pickle", "rb") as t:
#     tfidf_vect = pickle.load(t)
stop_ch1 = ['的', '了', '和', '而', '且', '1','2','3','4','5','6','7','8','9','10','11','12','1号','2号','3号','4号','5号','6号','7号','8号','9号','10号','11号','12号','一个','一','二','三','四','五','六','七','八','九','十','十一','十二','一号','二号','三号','四号','五号','六号','七号','八号','九号','十号','十一号','十二号']

tfidf_vect = TfidfVectorizer(min_df=.0025, max_df=0.75, max_features=None,stop_words=stop_ch1,token_pattern='[\u4e00-\u9fff]+',analyzer='word', ngram_range=(1, 3), norm='l2',use_idf=True)
X_train_tfidf = tfidf_vect.fit_transform(X_train)

clf_SVM_tfidf_b = LinearSVC(C=0.3,max_iter=100,penalty='l2',loss='squared_hinge')
model_SVM_tfidf_b = clf_SVM_tfidf_b.fit(X_train_tfidf, y_train)



In [186]:
predicted_SVM_tfidf_b = model_SVM_tfidf_b.predict(tfidf_vect.transform(df_test.segmented))
len(predicted_SVM_tfidf_b)

507

In [187]:
df_test['label'] = predicted_SVM_tfidf_b
df_test.head(10)

,timestamp,player,character,speech,segmented,syntax,label
3384,captain_election,5号,民,警上这么多人，只有四个人没竞选好吧，那我认警上应该是有两批狼左右,警上 这么 多人 ， 只有 四个 人 没 竞选 好 吧 ， 那 我认 警上 应该 是 有 两...,竞选 是 人 人 警上 有,attack
3385,captain_election,5号,民,呃很多村民竞选是不好的事，因为村民不在警下，其实给狼队提供了刀神的机会,呃 很多 村民 竞选 是 不好 的 事 ， 因为 村民 不 在 警下 ， 其实 给 狼队 提...,是 不在 呃 村民 事 警下,ph
3386,captain_election,5号,民,因为有些神是不敢上警的,因为 有些 神是 不敢 上警 的,是 核心关系 神 主谓关系 上警 主谓关系,ph
3387,captain_election,5号,民,当然你狼人可以赌反心态,当然 你 狼 人 可以 赌反 心态,赌 反 狼人 主谓关系 心态 主谓关系,ph
3388,captain_election,5号,民,所以我建议以后村民不要乱起跳，像我这种强民确实是下次不跳了,所以 我 建议 以后 村民 不要 乱 起跳 ， 像 我 这种 强民 确实 是 下次 不 跳 了,建议 核心关系 村民 强民 起跳 跳,defense
3389,captain_election,5号,民,5号第一个发言听不出来什么，但5号坚持竞选警长啊我过了,5 号 第一个 发言 听不出来 什么 ， 但 5 号 坚持 竞选 警长 啊 我 过 了,听不出来 坚持 发言 号 什么 竞选,ph
3390,captain_election,6号,预言家,6号是预言家嘿嘿,6 号 是 预言家 嘿嘿,是 核心关系 号 主谓关系 嘿嘿 主谓关系,protect
3391,captain_election,6号,预言家,上一轮验的是8号，是我的金水,上 一轮 验 的 是 8 号 ， 是 我 的 金水,是 是 验 主谓关系 号 金水,protect
3392,captain_election,6号,预言家,为什么要验他，因为8号是在我曾经落魄的时候，唯一站出来支持我的人,为什么 要验 他 ， 因为 8 号 是 在 我 曾经 落魄 的 时候 ， 唯一 站 出来 支...,验 是 号 我 他 站,ph
3393,captain_election,6号,预言家,给了我很多温暖,给 了 我 很多 温暖,很多 温暖 主谓关系 主谓关系 主谓关系 主谓关系,ph


In [188]:
display(df_test[['speech','syntax']])[:10]

,speech,syntax
3384,警上这么多人，只有四个人没竞选好吧，那我认警上应该是有两批狼左右,竞选 是 人 人 警上 有
3385,呃很多村民竞选是不好的事，因为村民不在警下，其实给狼队提供了刀神的机会,是 不在 呃 村民 事 警下
3386,因为有些神是不敢上警的,是 核心关系 神 主谓关系 上警 主谓关系
3387,当然你狼人可以赌反心态,赌 反 狼人 主谓关系 心态 主谓关系
3388,所以我建议以后村民不要乱起跳，像我这种强民确实是下次不跳了,建议 核心关系 村民 强民 起跳 跳
3389,5号第一个发言听不出来什么，但5号坚持竞选警长啊我过了,听不出来 坚持 发言 号 什么 竞选
3390,6号是预言家嘿嘿,是 核心关系 号 主谓关系 嘿嘿 主谓关系
3391,上一轮验的是8号，是我的金水,是 是 验 主谓关系 号 金水
3392,为什么要验他，因为8号是在我曾经落魄的时候，唯一站出来支持我的人,验 是 号 我 他 站
3393,给了我很多温暖,很多 温暖 主谓关系 主谓关系 主谓关系 主谓关系


TypeError: 'NoneType' object is not subscriptable

In [189]:
dic = {}
for x in list(df_test.syntax):
    for a in x.split()[2:4]:
        dic[a]=dic.get(a,0)+1
# sorted(dic, key=dic.get, reverse=True)

In [190]:
import re
def sentence_meaning(label,syntax,speech,player):
    attitudes = {}
    verb = syntax.split()[:2]
    subj = syntax.split()[2:4]
    obj  = syntax.split()[4:]
    
    if label=="to_check" or "验" in verb:
        if "警徽流" in speech:
            m = re.findall("[0-9]+(?=号)", speech)
            m = list(set(m))
            m = [a for a in m if a != player[:-1]]
            attitudes["警徽流"]=m
            attitudes["自认"]=["预言家"]
            
        else:
            m = re.findall("[0-9]+(?=号)", speech)
            m = list(set(m))
            m = [a for a in m if a != player[:-1]]
            attitudes["验"]=m
            
    elif label=="hang_on":
        m = re.findall("[0-9]+(?=号)", speech)
        m = list(set(m))
        m = [a for a in m if a != player[:-1]]
        attitudes["不确定"]=m
        
    elif label=="protect" or "捞" in verb or "保" in verb or "好人" in obj:
        m = re.findall("[0-9]+(?=号)", speech)
        m = list(set(m))
        m = [a for a in m if a != player[:-1]]
        attitudes["认好人"]=m
        for a in ["预言家","女巫","禁言长老","潜行者","金水","神","民"]:
            for s in speech.split("，"):
                if a in s:
                    m2 = re.findall("[0-9]+(?=号)", s)
                    m2 = list(set(m2))
                    if len(m)==1:
                        attitudes[a]= m
                    elif len(m)>=1 and a=="民":
                        attitudes[a]=m
            
    elif label=="attack":
        m = re.findall("[0-9]+(?=号)", speech)
        m = [a for a in m if a != player[:-1]]
        attitudes["认坏人"]=m
        
    elif label=="defense":
        if "我" in subj and "是" in verb:
            for a in ["预言家","女巫","潜行者"]:
                if a in obj:
                    attitudes["自认"]=attitudes.get("自认",[])+[a]
            for a in ["禁言长老","长老"]:
                if a in obj:
                    attitudes["自认"]=attitudes.get("自认",[])+["禁言长老"]
            for a in ["神民","强神","神","神牌"]:
                if a in obj:
                    attitudes["自认"]=attitudes.get("自认",[])+["神"]
            for a in ["民","闭眼玩家","平民"]:
                if a in obj:
                    attitudes["自认"]=attitudes.get("自认",[])+["民"]
            for a in ["银水","好人","金水","民及民以上"]:
                if a in obj:
                    attitudes["自认"]=attitudes.get("自认",[])+["好人"]
        else:
            m = re.findall("[0-9]+(?=号)", speech)
            m = [a for a in m if a != player[:-1]]
            attitudes["不对付"]=m
    else:
        for a in ["出","出局","聊爆","反水立警"]:
            if a in verb:
                m = re.findall("[0-9]+(?=号)", speech)
                m = [a for a in m if a != player[:-1]]
                attitudes["认坏人"]=m
    return attitudes

print("speech: 预言家先说一下警徽流，警徽流先验7号7号，再验5号5号\nsyntax: 说 号 预言家 主谓关系 警徽流 号\nlabel: to_check")
print("attitude dictionary: ",sentence_meaning("to_check","说 号 预言家 主谓关系 警徽流 号","预言家先说一下警徽流，警徽流先验7号7号，再验5号5号","2号"))        

speech: 预言家先说一下警徽流，警徽流先验7号7号，再验5号5号
syntax: 说 号 预言家 主谓关系 警徽流 号
label: to_check
attitude dictionary:  {'警徽流': ['5', '7'], '自认': ['预言家']}


In [191]:
# df_test into smaller df, each contains a timestamp
# each timestamp into smaller pieces, each a player

ply = list(df_test.player)
lab = list(df_test.label)
syn = list(df_test.syntax)
spe = list(df_test.speech)

abstract = []
for (label,syntax,speech,player) in zip(lab,syn,spe,ply):
    ab = sentence_meaning(label,syntax,speech,player)
    abstract.append(ab)
    
abstract[:5]

[{'认坏人': []}, {}, {}, {}, {'不对付': []}]

In [192]:
df_test['abstract']=abstract
df_test[['player','speech','syntax','label','abstract']].head()

,player,speech,syntax,label,abstract
3384,5号,警上这么多人，只有四个人没竞选好吧，那我认警上应该是有两批狼左右,竞选 是 人 人 警上 有,attack,{'认坏人': []}
3385,5号,呃很多村民竞选是不好的事，因为村民不在警下，其实给狼队提供了刀神的机会,是 不在 呃 村民 事 警下,ph,{}
3386,5号,因为有些神是不敢上警的,是 核心关系 神 主谓关系 上警 主谓关系,ph,{}
3387,5号,当然你狼人可以赌反心态,赌 反 狼人 主谓关系 心态 主谓关系,ph,{}
3388,5号,所以我建议以后村民不要乱起跳，像我这种强民确实是下次不跳了,建议 核心关系 村民 强民 起跳 跳,defense,{'不对付': []}


In [193]:
df_test.head()
df_test.abstract.dtype

dtype('O')

In [194]:
def unite_dict(df_test_grouped):
    list_of_dict = df_test_grouped.abstract
    big_dict={}
    for dic in list_of_dict:
        for key in dic:
            big_dict[key]=big_dict.get(key,[])+dic.get(key)
    return big_dict

# unite_dict([{'自认': ['预言家'], '警徽流': ['5', '7']},
#  {'认好人': ['12'], '金水': ['12']},
#  {},
#  {'认好人': ['2', '12']},
#  {'认好人': ['9'], '预言家': ['9']}])

In [195]:
g_ab = df_test.groupby(['timestamp', 'player']).apply(unite_dict).reset_index()
# with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', 1000):
#     display(g_ab)[:10]

g_ab.head()

,timestamp,player,0
0,captain_election,10号,"{'不对付': ['6'], '验': ['5', '7', '2', '4'], '认坏人..."
1,captain_election,11号,"{'不对付': [], '自认': ['神']}"
2,captain_election,12号,"{'认坏人': ['11', '10', '5', '10', '6', '10', '10..."
3,captain_election,1号,"{'不对付': [], '认好人': ['7'], '认坏人': ['6', '10', '..."
4,captain_election,5号,"{'认坏人': [], '不对付': []}"


In [196]:
# g_ab[('captain_election','2号')]  for groupby object who didn't reset index

In [197]:
def personal_perspective(abstract,player):
#     print(player)
    # character_distribution has length 13, every index directly corresponds to a player
    character_distribution=["UNK", "UNK", "UNK", "UNK", "UNK", "UNK", "UNK", "UNK", "UNK", "UNK", "UNK", "UNK", "UNK"]
    if '自认' in abstract:
        if len(abstract['自认'])==1:
            character_distribution[int(player[:-1])]=abstract['自认'][-1]
        else:
            print(player,'自认',abstract['自认'])
            character_distribution[int(player[:-1])]=abstract['自认'][-1]
    
    if '警徽流' in abstract:
        for a in abstract['警徽流']:
            character_distribution[int(a)]="验"
    if '验' in abstract:
        for a in abstract['验']:
            character_distribution[int(a)]="验"
    if '不对付' in abstract:
        for a in abstract['不对付']:
            character_distribution[int(a)]='不对付'
            
    if '不确定' in abstract: # which order is better ? 
        for a in abstract['不确定']:
            character_distribution[int(a)]="不确定"
    
    if '认坏人' in abstract:
        for a in set(abstract['认坏人']):
            if '认好人' in abstract:
                if a not in abstract['认好人']:
                    character_distribution[int(a)]="认坏人"
                else:
                    bad = abstract['认坏人'].count(a)
                    good= abstract['认好人'].count(a)
                    if bad>=good:
                        character_distribution[int(a)]="认坏人"
                    else:
                        character_distribution[int(a)]="认好人"
            else:
                character_distribution[int(a)]="认坏人"
                    
            
    if '认好人' in abstract:
        for a in set(abstract['认好人']):
            if '认坏人' in abstract:
                if a not in abstract['认坏人']:
                    character_distribution[int(a)]="'认好人'"
                else:
                    bad = abstract['认坏人'].count(a)
                    good= abstract['认好人'].count(a)
                    if bad>=good:
                        character_distribution[int(a)]="认坏人"
                    else:
                        character_distribution[int(a)]="认好人"
            else:
                character_distribution[int(a)]="认好人"
            
            
    if '金水' in abstract:
        character_distribution[int(abstract['金水'][-1])]='金水'
    
    for a in ["预言家","女巫","禁言长老","潜行者"]:
        if a in abstract:
            character_distribution[int(abstract[a][-1])]=a
    
    return character_distribution

In [198]:
# ab = g_ab[('captain_election','2号')]
# personal_perspective(ab,'2号')

In [199]:

g_ab['pattern']=g_ab.apply(lambda x: personal_perspective(x[0],x.player),axis=1)


12号 自认 ['预言家', '神']


In [200]:
g_ab.head()

,timestamp,player,0,pattern
0,captain_election,10号,"{'不对付': ['6'], '验': ['5', '7', '2', '4'], '认坏人...","[UNK, UNK, 验, UNK, 验, 验, 认坏人, 验, UNK, UNK, UNK..."
1,captain_election,11号,"{'不对付': [], '自认': ['神']}","[UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, ..."
2,captain_election,12号,"{'认坏人': ['11', '10', '5', '10', '6', '10', '10...","[UNK, UNK, UNK, UNK, UNK, 认坏人, 认坏人, '认好人', UNK..."
3,captain_election,1号,"{'不对付': [], '认好人': ['7'], '认坏人': ['6', '10', '...","[UNK, UNK, UNK, UNK, UNK, UNK, 认坏人, '认好人', UNK..."
4,captain_election,5号,"{'认坏人': [], '不对付': []}","[UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, UNK, ..."


In [201]:
g_ab['pattern'][2]

['UNK',
 'UNK',
 'UNK',
 'UNK',
 'UNK',
 '认坏人',
 '认坏人',
 "'认好人'",
 'UNK',
 '认坏人',
 '预言家',
 '认坏人',
 '神']

In [202]:
import json

In [203]:
with open("s1e102_behavior.json") as f:
    behavior_record = json.load(f)

In [204]:
proba_wolf = [0.33]*13

In [205]:
# receive count
from collections import Counter
def receive(list_of_pattern):
    c=[]
    for player in zip(*list_of_pattern):
        c.append(Counter(player)) 

    return c
                
                
p=g_ab.groupby('timestamp')
abpattern=p['pattern'].apply(receive).reset_index()
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', 1000):
    display(abpattern)

,timestamp,pattern
0,captain_election,"[{'UNK': 8}, {'UNK': 8}, {'验': 1, 'UNK': 6, '认坏人': 1}, {'UNK': 8}, {'验': 1, 'UNK': 7}, {'验': 1, 'UNK': 5, '认坏人': 2}, {'认坏人': 3, 'UNK': 3, '预言家': 2}, {'验': 1, 'UNK': 3, ''认好人'': 2, '民': 1, '认坏人': 1}, {'UNK': 7, '金水': 1}, {'UNK': 5, '认坏人': 1, '不确定': 1, '预言家': 1}, {'UNK': 3, '预言家': 1, '认坏人': 4}, {'UNK': 6, '神': 1, '认坏人': 1}, {'UNK': 5, '神': 1, '验': 1, '认坏人': 1}]"
1,day1_lastwords,"[{'UNK': 1}, {'认坏人': 1}, {'UNK': 1}, {'认坏人': 1}, {'认坏人': 1}, {'UNK': 1}, {'UNK': 1}, {'UNK': 1}, {'UNK': 1}, {''认好人'': 1}, {'认坏人': 1}, {'UNK': 1}, {'UNK': 1}]"
2,day1_speech,"[{'UNK': 12}, {'认坏人': 4, 'UNK': 8}, {'认坏人': 2, 'UNK': 9, '不对付': 1}, {'认坏人': 4, 'UNK': 8}, {'认坏人': 1, 'UNK': 10, ''认好人'': 1}, {'认坏人': 1, 'UNK': 9, '不对付': 1, ''认好人'': 1}, {'认坏人': 9, 'UNK': 3}, {''认好人'': 1, 'UNK': 7, '认坏人': 4}, {''认好人'': 1, '认坏人': 4, 'UNK': 7}, {'认坏人': 7, 'UNK': 5}, {'预言家': 1, '认坏人': 8, '不对付': 1, 'UNK': 2}, {'UNK': 9, '认坏人': 2, ''认好人'': 1}, {'认坏人': 5, 'UNK': 6, '不对付': 1}]"
3,day2_speech,"[{'UNK': 2}, {'UNK': 2}, {'UNK': 1, '认坏人': 1}, {'UNK': 1, '认坏人': 1}, {'UNK': 2}, {'UNK': 2}, {'UNK': 2}, {'UNK': 2}, {'UNK': 2}, {'UNK': 2}, {'UNK': 1, '认坏人': 1}, {'UNK': 2}, {'UNK': 2}]"
4,day3_speech,"[{'UNK': 4}, {'UNK': 3, '认坏人': 1}, {'UNK': 3, '不对付': 1}, {'UNK': 3, '认坏人': 1}, {'UNK': 4}, {'UNK': 3, ''认好人'': 1}, {'UNK': 3, '认坏人': 1}, {'UNK': 4}, {'UNK': 3, '认坏人': 1}, {'UNK': 3, '认坏人': 1}, {'UNK': 3, '认坏人': 1}, {'UNK': 4}, {'UNK': 4}]"
5,day4_speech,"[{'UNK': 2}, {'认坏人': 1, 'UNK': 1}, {'认坏人': 1, 'UNK': 1}, {'UNK': 2}, {'UNK': 2}, {'UNK': 2}, {'认坏人': 1, 'UNK': 1}, {'认坏人': 1, 'UNK': 1}, {'UNK': 2}, {'UNK': 2}, {'认坏人': 1, 'UNK': 1}, {'UNK': 2}, {'UNK': 2}]"
6,extra_pk,"[{'UNK': 10}, {'UNK': 7, ''认好人'': 1, '认坏人': 2}, {'UNK': 7, ''认好人'': 1, '认坏人': 2}, {'UNK': 7, ''认好人'': 2, '认坏人': 1}, {'UNK': 7, '预言家': 1, '认坏人': 2}, {'UNK': 8, '认坏人': 2}, {'认坏人': 8, '金水': 1, '认好人': 1}, {'UNK': 8, '认坏人': 2}, {'UNK': 6, '认坏人': 1, ''认好人'': 2, '金水': 1}, {'UNK': 5, '认坏人': 4, '不确定': 1}, {'预言家': 2, '认坏人': 8}, {'UNK': 6, '认坏人': 3, ''认好人'': 1}, {'UNK': 2, '认坏人': 7, ''认好人'': 1}]"
7,pk_speech,"[{'UNK': 2}, {'UNK': 2}, {'UNK': 2}, {'UNK': 2}, {'UNK': 2}, {'UNK': 1, '认坏人': 1}, {'认坏人': 1, 'UNK': 1}, {'UNK': 2}, {'UNK': 2}, {'UNK': 2}, {'预言家': 1, '认坏人': 1}, {'金水': 1, 'UNK': 1}, {'认坏人': 2}]"


In [206]:
# infty = 10**6

# # scapegoat : the wolf that is abandoned by the others

# def find_scapegoat(wolf_list,speech_summary,votes):
#     scapegoat = 0
#     scapelikelihood = 0
#     for w in wolf_list:
#         likelihood = 0
#         for w1 in wolf_list:
#             if w1!=w and speech_summary[w1][w]=='认坏人':
#                 likelihood += 1
#             if w1!=w and votes[str(w1)]==str(w):
#                 likelihood += 2
#         if likelihood > scapelikelihood:
#             scapegoat = w
#     return(scapegoat)                

# # score of a hypothesis : bigger = less likely

# alpha = 1
# alphavote = 2
# beta = 0.3
# gamma = 1
# gammavote = 2
# delta = 1
# absolute_coef = 3


# def give_instant_score(wolf_list,villager_list,speech_summary,list_receive,votes,character_distribution):
#     score = 0
#     for v in villager_list:
#         if character_distribution[v]=="werewolf":
#             return(infty)
#     wolf_coef = {} # gives the multiplicative coefficient of a wolf, greater if the wolf is confirmed
#     for w in wolf_list:
#         if character_distribution[w]=="werewolf":
#             wolf_coef[w] = absolute_coef
#         else:
#             wolf_coef[w] = 1
#     scapegoat = find_scapegoat(wolf_list,speech_summary,votes)
#     for i in wolf_list:
#         for j in wolf_list:
#             if j!=scapegoat and speech_summary[i][j]=='认坏人':
#                 score += alpha*wolf_coef[w]
#             if j!=scapegoat and votes[str(i)]==str(j):
#                 score += alphavote*wolf_coef[w]
#         for j in villager_list:
#             if speech_summary[i][j]=='认好人':
#                 score += beta*wolf_coef[w]
#     probably_good = find_good(list_receive)
#     probably_bad = find_bad(list_receive)
#     for i in villager_list:
#         for j in probably_good:
#             if speech_summary[i][j]=='认坏人':
#                 score += gamma
#             if votes[str(i)]==str(j):
#                 score += gammavote
#         for j in probably_bad:
#             if speech_summary[i][j]=='认好人':
#                 score += delta
#     return(score)

# def compute_all_instant_scores(speech_summary,votes,character_distribution):
#     #input:
#     #speech_summary: 2D table such that speech_summary[i] is player i's pattern for the current turn. We want only 'UNK','认好人' or '认坏人'. 
#     #votes: dictionary of votes for the current turn, as in behavior json
#     #character_distribution: list of character distribution, as in behavior json
#     list_receive = [{}]
#     for i in range(1,13):
#         d = {'认好人':0,'认坏人':0}
#         for j in range(1,13):
#             if speech_summary[j][i]=='认好人':
#                 d['认好人'] += 1
#             if speech_summary[j][i]=='认坏人':
#                 d['认坏人'] += 1
#         list_receive.append(d)
#     scores = {}
#     for c in itertools.combinations(range(1,13),4):
#         wolf_list = list(c)
#         villager_list = []
#         for i in range(1,13):
#             if i not in wolf_list:
#                 villager_list.append(i)
#         scores[set(wolf_list)] = give_instant_score(wolf_list,villager_list,speech_summary,list_receive,player_alive,votes,character_distribution)
#     return(scores)
        
        
    
    
        

In [207]:
def time_to_timestamp(t):
    if t==0:
        return 'captain_election'
    if t==0.2:
        return 'pk_speech'
    if t==0.7:
        return 'extra_pk'
    else:
        n = t//1
        if t-n==0.2:
            return 'day'+str(int(n))+'_lastwords'
        else:
            return 'day'+str(int(n))+'_speech'

def timestamp_to_time(timestamp):
    if timestamp=='captain_election':
        return 0
    elif timestamp=='pk_speech':
        return 0.2
    elif timestamp=='extra_pk':
        return 0.7
    elif timestamp.endswith('speech'):
        return int(timestamp[3])
    elif timestamp.endswith('lastwords'):
        return int(timestamp[3])+0.2

In [208]:
def get_speeches(g_ab,binary=True):
    #binary=True: gives everything in terms of 'UNK', '认好人' and '认坏人'
    speeches = {}
    for index,row in g_ab.iterrows():
        speeches[timestamp_to_time(row.timestamp)] = [['UNK' for i in range(13)] for j in range(13)]
    for index,row in g_ab.iterrows():
        player,t,pattern = int(row.player[:-1]),timestamp_to_time(row.timestamp),row.pattern
        if binary:
            goodid = ["神","民","金水","预言家","女巫","潜行者","禁言长老","猎人","白痴","守卫",'好人','认好人']
            pattern = ['认好人' if a in goodid else a for a in pattern]
        speeches[t][player] = pattern
    return speeches

def get_votes(behavior_record):
    votes = {}
    for dic in behavior_record:
        if dic['stage']!='initial':
            t = timestamp_to_time(dic['stage'])
            votes[t] = dic['dispositive']['votes'][0]
    return votes

def get_character_distribution(behavior_record,time):
    timestamp = time_to_timestamp(time)
    for dic in behavior_record:
        if dic['stage']==timestamp:
            character_distribution = dic['dispositive']['character_distribution']
    return character_distribution

In [209]:
# scapegoat : the wolf that is abandoned by the others

def find_scapegoat(wolf_list,speeches,votes,t):
    scapegoat = 0
    scapelikelihood = 0
    for w in wolf_list:
        likelihood = 0
        for w1 in wolf_list:
            if w1!=w and speeches[t][w1][w]=='认坏人':
                likelihood += 1
            if w1!=w and t>=1 and str(w1) in votes[t] and votes[t][str(w1)]==str(w):
                likelihood += 2
            if w1!=w and t<1 and str(w1) in votes[t] and votes[t][str(w1)]==str(w):
                likelihood -= 3
        if likelihood > scapelikelihood:
            scapelikehood = likelihood
            scapegoat = w
    return scapegoat

def find_bad(list_receive,bad_threshold=5):
    bad = []
    for i in range(1,len(list_receive)):
        dic=list_receive[i]
        if dic.get('认坏人')>=bad_threshold:
            bad.append(i)
    return bad

def find_good(list_receive,good_threshold=3):
    good = []
    for i in range(1,len(list_receive)):
        dic=list_receive[i]
        if dic.get('认好人')>=good_threshold:
            good.append(i)
    return good

# score of a hypothesis : bigger = less likely

alpha = 1
alphavote = 2
beta = 0.3
betavote = 0.5
gamma = 1
gammavote = 2
delta = 1
deltavote = 2
absolute_coef = 3
time_decay = 1.6

def give_score(wolf_list,villager_list,speeches,list_receive,votes,character_distribution,times_list,
               alpha=1,alphavote=2,
               beta=0.3,betavote=0.5,
               gamma=1,gammavote=2,
               delta=1,deltavote=2,
               absolute_coef=3,
               infty=10**6,
               time_decay=1.6,
               bad_threshold=5,
               good_threshold=3):
    score = 0
    for v in villager_list:
        if character_distribution[v]=="werewolf":
            return(infty)
    wolf_coef = {} # gives the multiplicative coefficient of a wolf, greater if the wolf is confirmed
    for w in wolf_list:
        if character_distribution[w]=="werewolf":
            wolf_coef[w] = absolute_coef
        else:
            wolf_coef[w] = 1
    scapegoat = find_scapegoat(wolf_list,speeches,votes,max(times_list))
    for t in times_list:
        instant_score = 0
        for i in wolf_list:
            for j in wolf_list:
                if j!=scapegoat and speeches[t][i][j]=='认坏人':
                    instant_score += alpha*wolf_coef[w]
                if j!=scapegoat and t>=1 and str(i) in votes[t] and votes[t][str(i)]==str(j): #t>=1 because captain election votes have opposite meaning as execution vote
                    instant_score += alphavote*wolf_coef[w]
            for j in villager_list:
                if speeches[t][i][j]=='认好人':
                    instant_score += beta*wolf_coef[w]
                if j!=scapegoat and t<1 and str(i) in votes[t] and votes[t][str(i)]==str(j): #t<1 for the same reason as above
                    instant_score += betavote*wolf_coef[w]
        probably_good = find_good(list_receive[t],good_threshold)
        probably_bad = find_bad(list_receive[t],bad_threshold)
        for i in villager_list:
            for j in probably_good:
                if speeches[t][i][j]=='认坏人':
                    instant_score += gamma
                if t>=1 and str(i) in votes[t] and votes[t][str(i)]==str(j): #again captain election != execution vote
                    instant_score += gammavote
            for j in probably_bad:
                if speeches[t][i][j]=='认好人':
                    instant_score += delta
                if t<1 and str(i) in votes[t] and votes[t][str(i)]==str(j): #again captain election != execution vote
                    instant_score += deltavote
        score += instant_score*(time_decay**t)
    return(score)

def compute_all_scores(g_ab,behavior_record,current_timestamp='end',
                       alpha=1,alphavote=2,
                       beta=0.3,betavote=0.5,
                       gamma=1,gammavote=2,
                       delta=1,deltavote=2,
                       absolute_coef=3,
                       infty=10**6,
                       time_decay=1.6,
                       bad_threshold=5,
                       good_threshold=3):
    #input:
    #speeches: 3D table such that speeches[t][i] is player i's pattern for the turn t. We want only 'UNK','认好人' or '认坏人'. 
    #votes: list of dictionaries, such that votes[t] is the dictionary of votes for the turn t as in behavior json
    #character_distribution: list of character distribution, as in behavior json
    speeches = get_speeches(g_ab)
    votes = get_votes(behavior_record)
    if current_timestamp!='end':
        current_time = timestamp_to_time(current_timestamp)
    else:
        current_time = 0
    times_list = [] #unordered!
    for dic in behavior_record:
        if dic['stage']!='initial':
            t = timestamp_to_time(dic['stage'])
            if current_timestamp=='end':
                times_list.append(t)
                if current_time<t:
                    current_time = t
            elif t<=current_time:
                times_list.append(t)
    #at this point times_list has all times up to current_time, and current_time is the biggest one if current_timestamp=='end'
    character_distribution = get_character_distribution(behavior_record,current_time)
    list_receive = {}
    for t in times_list:
        instant_list_receive = [{}]
        for i in range(1,13):
            d = {'认好人':0,'认坏人':0}
            for j in range(1,13):
                if speeches[t][j][i]=='认好人':
                    d['认好人'] += 1
                if speeches[t][j][i]=='认坏人':
                    d['认坏人'] += 1
            instant_list_receive.append(d)
        list_receive[t] = instant_list_receive
    scores = {}
    for wolf_tuple in itertools.combinations(range(1,13),4):
        wolf_list = list(wolf_tuple)
        villager_list = []
        for i in range(1,13):
            if i not in wolf_list:
                villager_list.append(i)
        scores[wolf_tuple] = give_score(wolf_list,villager_list,speeches,list_receive,votes,character_distribution,times_list)
    return(scores)

In [210]:
def guess_wolfteam(timestamp, g_ab=g_ab,behavior_record=behavior_record):
    scores = compute_all_scores(g_ab,behavior_record,timestamp)
    l = sorted(scores,key=scores.get)
#     print(timestamp)
#     for x in l[:10]:
    for x in l[:1]:
        print(timestamp,x,scores[x])
        
guess_wolfteam('end')

end (1, 9, 10, 11) 36.67341032423715


In [211]:
def get_timestamps(behavior_record=behavior_record):
    timestamps = []
    for dic in behavior_record:
        timestamps.append(dic['stage'])
    return timestamps[1:]+['end']

In [212]:
for timestamp in get_timestamps():
    print(guess_wolfteam(timestamp))

captain_election (1, 2, 3, 5) 0.0
None
pk_speech (2, 3, 10, 11) 0.0
None
extra_pk (2, 4, 10, 11) 20.843720786636244
None
day1_speech (1, 4, 10, 11) 24.254139400860495
None
day1_lastwords (1, 2, 4, 11) 27.78500931893818
None
day2_speech (1, 4, 10, 11) 24.254139400860495
None
day3_speech (8, 9, 10, 11) 34.53431851722659
None
day4_speech (1, 9, 10, 11) 36.67341032423715
None
end (1, 9, 10, 11) 36.67341032423715
None


In [213]:
def wolf_probability(g_ab,behavior_record,current_timestamp='end',N=10): #N: we look at the top N quadruples
    scores = compute_all_scores(g_ab,behavior_record,current_timestamp)
    l = sorted(scores,key=scores.get)
    probas = [0 for i in range(12)]
    for quadruple in l[:N]:
        a,b,c,d = quadruple
        probas[a-1] += 1/N
        probas[b-1] += 1/N
        probas[c-1] += 1/N
        probas[d-1] += 1/N
    return probas

In [214]:
wolf_probability(g_ab,behavior_record,'end',10) # This is the final guess

[0.9999999999999999,
 0.2,
 0.2,
 0.2,
 0.1,
 0.1,
 0.1,
 0.1,
 0.8999999999999999,
 0.8999999999999999,
 0.1,
 0.1]

In [215]:



for timestamp in get_timestamps():
    print(timestamp+':')
    print([round(x,3) for x in wolf_probability(g_ab,behavior_record,timestamp,10)])

captain_election:
[1.0, 1.0, 0.4, 0, 0.4, 0, 0, 0, 0.4, 0.4, 0.4, 0]
pk_speech:
[0, 0.6, 0.4, 0.3, 0.5, 0.5, 0.1, 0.5, 0.2, 0.5, 0.4, 0]
extra_pk:
[0.4, 0.4, 0.4, 1.0, 0.4, 0, 0, 0, 0, 0.4, 1.0, 0]
day1_speech:
[0.6, 0.5, 0.3, 0.8, 0.1, 0.1, 0, 0.1, 0.2, 0.4, 0.9, 0]
day1_lastwords:
[0.7, 0.6, 0.6, 0.7, 0.7, 0, 0, 0, 0, 0, 0.7, 0]
day2_speech:
[0.2, 0.2, 0.1, 0.9, 0.2, 0, 0.1, 0.1, 0.1, 1.0, 0.9, 0.2]
day3_speech:
[0.1, 0.1, 0.1, 0.2, 0.1, 0, 0.1, 0.2, 1.0, 1.0, 0.8, 0.3]
day4_speech:
[1.0, 0.2, 0.2, 0.2, 0.1, 0.1, 0.1, 0.1, 0.9, 0.9, 0.1, 0.1]
end:
[1.0, 0.2, 0.2, 0.2, 0.1, 0.1, 0.1, 0.1, 0.9, 0.9, 0.1, 0.1]


In [151]:
map

2.35